In [1]:
# Customer lifecycle cleaning demo
# --------------------------------
# Goal: take a messy customer-level lifecycle & revenue export and turn it into
# an analysis-ready table a senior leader could use for LTV, churn, and segmentation work.

import os
import numpy as np
import pandas as pd

# --------------------------------------------------------------------
# 1. Paths & data load
# --------------------------------------------------------------------

RAW_PATH = "../../../data/cleaning_demo/customer_lifecycle_raw.csv"
OUTPUT_DIR = "../../../outputs/data/cleaning_demo"
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PATH = os.path.join(OUTPUT_DIR, "customer_lifecycle_cleaned.csv")

df_raw = pd.read_csv(RAW_PATH)
print("Raw shape:", df_raw.shape)
df_raw.head()

# Work on a copy so we can always refer back to df_raw if needed
df = df_raw.copy()

# --------------------------------------------------------------------
# 2. Parse dates & fix basic types
# --------------------------------------------------------------------

date_cols = ["signup_date", "first_purchase_date", "last_purchase_date"]
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce", infer_datetime_format=True)

numeric_cols = [
    "total_revenue",
    "num_orders",
    "avg_order_value",
    "days_since_last_purchase",
    "nps_last_6mo",
]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# --------------------------------------------------------------------
# 3. Standardize categorical fields
# --------------------------------------------------------------------

def normalize_channel(x):
    if pd.isna(x) or str(x).strip() == "":
        return np.nan
    t = str(x).strip().lower()
    if "paid" in t or "ppc" in t:
        return "Paid Search"
    if "organic" in t or "seo" in t or "org " in t:
        return "Organic"
    if "email" in t or "eml" in t:
        return "Email"
    if "ref" in t:
        return "Referral"
    if "social" in t or "fb" in t or "ig" in t:
        return "Social"
    return "Other"

def normalize_region(x):
    if pd.isna(x) or str(x).strip() == "":
        return np.nan
    t = str(x).strip().lower()
    if "north" in t or t in {"ne", "n.e."}:
        return "Northeast"
    if "mid" in t or "mw" in t:
        return "Midwest"
    if "south" in t or "se/" in t or "sw" in t:
        return "South"
    if "west" in t or t in {"w.", "pac west"}:
        return "West"
    return "Other"

def normalize_tier(x):
    if pd.isna(x) or str(x).strip() == "":
        return "None"
    t = str(x).strip().lower()
    if t.startswith("gold") or t == "gld":
        return "Gold"
    if t.startswith("silver") or t == "slvr":
        return "Silver"
    if t.startswith("bronze") or t == "brz" or t == "brnze":
        return "Bronze"
    if t in {"none", "na"}:
        return "None"
    return "None"

def normalize_opt_in(x):
    if pd.isna(x):
        return np.nan
    t = str(x).strip().lower()
    if t in {"true", "y", "yes", "1"}:
        return True
    if t in {"false", "n", "no", "0", ""}:
        return False
    return np.nan

def normalize_status(x):
    if pd.isna(x) or str(x).strip() == "":
        return np.nan
    t = str(x).strip().lower()
    if t in {"active", "act", "current"}:
        return "Active"
    if "churn" in t or t in {"lost", "chrn"}:
        return "Churned"
    if "risk" in t:
        return "At Risk"
    return "Other"

df["acquisition_channel"] = df["acquisition_channel"].apply(normalize_channel)
df["region"] = df["region"].apply(normalize_region)
df["membership_tier"] = df["membership_tier"].apply(normalize_tier)
df["email_opt_in"] = df["email_opt_in"].apply(normalize_opt_in)
df["status"] = df["status"].apply(normalize_status)

# Strip whitespace from industry labels and standardize capitalization
df["industry"] = (
    df["industry"]
    .astype("string")
    .str.strip()
    .replace("", pd.NA)
    .str.title()
)

# --------------------------------------------------------------------
# 4. Handle numeric anomalies (revenue, NPS, days since last purchase)
# --------------------------------------------------------------------

# Negative revenue is almost always a data error in this context -> set to NaN
df["total_revenue"] = df["total_revenue"].where(df["total_revenue"] >= 0, np.nan)

# Recalculate avg_order_value from cleaned revenue & num_orders
df["avg_order_value"] = df["total_revenue"] / df["num_orders"]

# Valid NPS scores are 0–10; treat anything else (11, 12, 99, etc.) as missing
df["nps_last_6mo"] = df["nps_last_6mo"].where(
    df["nps_last_6mo"].between(0, 10), np.nan
)

# Recalculate days_since_last_purchase from last_purchase_date for consistency
reference_date = pd.to_datetime("2024-01-01")
df["days_since_last_purchase"] = (
    (reference_date - df["last_purchase_date"]).dt.days
)
df.loc[df["last_purchase_date"].isna(), "days_since_last_purchase"] = np.nan

# --------------------------------------------------------------------
# 5. Deduplicate customers
# --------------------------------------------------------------------
# In the raw export, some customers appear multiple times.
# We keep the most recent, highest-revenue record per customer_id.

df_sorted = df.sort_values(
    by=["customer_id", "last_purchase_date", "total_revenue"],
    ascending=[True, False, False],
)

df_dedup = df_sorted.drop_duplicates(subset="customer_id", keep="first")
print("After de-duplication:", df_dedup.shape)

# --------------------------------------------------------------------
# 6. Derive business-friendly fields
# --------------------------------------------------------------------

# Customer "age" in days since signup
df_dedup["customer_age_days"] = (reference_date - df_dedup["signup_date"]).dt.days

# Flag: recently active (e.g., purchased in last 90 days)
df_dedup["is_active_90d"] = df_dedup["days_since_last_purchase"] <= 90

# Simple revenue banding for segmentation
def revenue_band(x):
    if pd.isna(x):
        return "Unknown"
    if x < 500:
        return "< $500"
    if x < 2000:
        return "$500–$2k"
    if x < 5000:
        return "$2k–$5k"
    if x < 10000:
        return "$5k–$10k"
    return ">$10k"

df_dedup["revenue_band"] = df_dedup["total_revenue"].apply(revenue_band)

# --------------------------------------------------------------------
# 7. Final column ordering & save
# --------------------------------------------------------------------

cols_order = [
    "customer_id",
    "signup_date",
    "first_purchase_date",
    "last_purchase_date",
    "customer_age_days",
    "days_since_last_purchase",
    "is_active_90d",
    "status",
    "acquisition_channel",
    "region",
    "industry",
    "membership_tier",
    "email_opt_in",
    "num_orders",
    "total_revenue",
    "avg_order_value",
    "revenue_band",
    "nps_last_6mo",
]

# Keep only columns that actually exist (defensive)
cols_order = [c for c in cols_order if c in df_dedup.columns]
df_clean = df_dedup[cols_order].copy()

print("Cleaned shape:", df_clean.shape)
df_clean.head()

df_clean.to_csv(OUTPUT_PATH, index=False)
print("Saved cleaned dataset to:", OUTPUT_PATH)


Raw shape: (1280, 15)
After de-duplication: (1200, 15)
Cleaned shape: (1200, 18)
Saved cleaned dataset to: ../../../outputs/data/cleaning_demo/customer_lifecycle_cleaned.csv


/var/folders/5v/5z1dw78974g093k15vck26lm0000gn/T/ipykernel_6965/2369186487.py:32: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(df[col], errors="coerce", infer_datetime_format=True)
/var/folders/5v/5z1dw78974g093k15vck26lm0000gn/T/ipykernel_6965/2369186487.py:32: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(df[col], errors="coerce", infer_datetime_format=True)
/var/folders/5v/5z1dw78974g093k15vck26lm0000gn/T/ipykernel_6965/2369186487.py:32: UserWarning: The argument 'infer_datetime_format' is deprecated an

In [2]:
from ydata_profiling import ProfileReport
import os

# Create a profiling report on the cleaned dataset
profile = ProfileReport(
    df_clean,
    title="Customer Lifecycle – Cleaning Demo",
    explorative=True
)

report_dir = "../../../docs/cleaning_demo"
os.makedirs(report_dir, exist_ok=True)
report_path = os.path.join(report_dir, "index.html")

profile.to_file(report_path)
print("Saved profile report to:", report_path)


/Users/sharongrant/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/sharongrant/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Export report to file: 100%|█████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 551.01it/s]

Saved profile report to: ../../../docs/cleaning_demo/index.html
